In [8]:
import numpy as np
import random
from tensorflow import keras
# Ensure the data is reshaped correctly
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Normalize and split the data
X_train_full, X_test = X_train_full / 255.0, X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


X_train = X_train.reshape(-1, 28, 28)  # Reshape if necessary
X_valid = X_valid.reshape(-1, 28, 28)  # Reshape if necessary


# Step 1: Define search space
def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        individual = {
            "neurons1": random.randint(32, 512),
            "neurons2": random.randint(32, 512),
            "learning_rate": 10 ** random.uniform(-4, -2)  # Log-scale
        }
        population.append(individual)
    return population

# Step 2: Create the model
def create_model(neurons1, neurons2, lr):
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),  # Adjust input shape for MNIST
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(neurons1, kernel_initializer='lecun_normal', activation='selu'),
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(neurons2, kernel_initializer='lecun_normal', activation='selu'),
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(10, activation="softmax")  # 10 classes for digit classification
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                  loss="sparse_categorical_crossentropy", 
                  metrics=["accuracy"])
    return model


# Step 3: Fitness function

def fitness(individual, X_train, y_train, X_valid, y_valid):
    

    model = create_model(individual["neurons1"], individual["neurons2"], individual["learning_rate"])
    early_stopping = keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid),
                    batch_size=32, verbose=0, callbacks=[early_stopping])
    # history = model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid), batch_size=32, verbose=0)
    val_loss = history.history["val_loss"][-1]  # Use final validation loss as fitness
    val_accuracy = history.history["val_accuracy"][-1]  # Final validation accuracy
    return val_loss, val_accuracy
    
# Step 4: Selection (Tournament Selection)
def select_parents(population, fitness_scores):
    parents = random.choices(population, weights=1/np.array(fitness_scores), k=2)  # Lower loss = higher fitness
    return parents

# Step 5: Crossover
def crossover(parent1, parent2):
    child = {
        "neurons1": random.choice([parent1["neurons1"], parent2["neurons1"]]),
        "neurons2": random.choice([parent1["neurons2"], parent2["neurons2"]]),
        "learning_rate": random.choice([parent1["learning_rate"], parent2["learning_rate"]])
    }
    return child

# Step 6: Mutation
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        individual["neurons1"] = random.randint(32, 512)
    if random.random() < mutation_rate:
        individual["neurons2"] = random.randint(32, 512)
    if random.random() < mutation_rate:
        individual["learning_rate"] = 10 ** random.uniform(-4, -2)
    return individual

# Step 7: Genetic Algorithm
def genetic_algorithm(X_train, y_train, X_valid, y_valid, generations, pop_size, mutation_rate):
    population = initialize_population(pop_size)
    for generation in range(generations):
        print(f"Generation {generation+1}/{generations}")
        
        # Evaluate fitness
        results = [fitness(ind, X_train, y_train, X_valid, y_valid) for ind in population]
        fitness_scores = [result[0] for result in results]  # Extract val_loss
        accuracy_scores = [result[1] for result in results]  # Extract val_accuracy
        
        # Log the best results of the current generation
        best_loss = min(fitness_scores)
        best_accuracy = max(accuracy_scores)
        print(f"Best fitness (val_loss): {best_loss}")
        print(f"Best accuracy (val_accuracy): {best_accuracy}")
        
        # Select next generation
        new_population = []
        for _ in range(pop_size // 2):  # Each iteration produces 2 children
            parent1, parent2 = select_parents(population, fitness_scores)
            child1 = mutate(crossover(parent1, parent2), mutation_rate)
            child2 = mutate(crossover(parent1, parent2), mutation_rate)
            new_population.extend([child1, child2])
        
        population = new_population
    
    # Return the best hyperparameters
    best_index = np.argmin(fitness_scores)
    # print(f"Final Best Accuracy (val_accuracy): {accuracy_scores[best_index]}")
    return population[best_index], best_loss, accuracy_scores[best_index]

#  Run Genetic Algorithm
best_hyperparameters, best_loss, best_accuracy = genetic_algorithm(
    X_train, y_train, X_valid, y_valid, 
    generations=5, 
    pop_size=5, 
    mutation_rate=0.1
)

print("Best Hyperparameters:", best_hyperparameters)
print(f"Best Validation Loss: {best_loss}")
print(f"Best Validation Accuracy: {best_accuracy}")

Generation 1/5
Best fitness (val_loss): 0.09121539443731308
Best accuracy (val_accuracy): 0.9747999906539917
Generation 2/5
Best fitness (val_loss): 0.08174171298742294
Best accuracy (val_accuracy): 0.9765999913215637
Generation 3/5
Best fitness (val_loss): 0.07699450850486755
Best accuracy (val_accuracy): 0.9800000190734863
Generation 4/5
Best fitness (val_loss): 0.08287367224693298
Best accuracy (val_accuracy): 0.9783999919891357
Generation 5/5
Best fitness (val_loss): 0.0943819060921669
Best accuracy (val_accuracy): 0.9724000096321106
Best Hyperparameters: {'neurons1': 135, 'neurons2': 204, 'learning_rate': 0.0009977066121945018}
Best Validation Loss: 0.0943819060921669
Best Validation Accuracy: 0.9715999960899353


In [2]:
import numpy as np
import random
from tensorflow import keras
# Ensure the data is reshaped correctly
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Normalize and split the data
X_train_full, X_test = X_train_full / 255.0, X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


X_train = X_train.reshape(-1, 28, 28)  # Reshape if necessary
X_valid = X_valid.reshape(-1, 28, 28)  # Reshape if necessary

# Step 1: Define search space
def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        individual = {
            "neurons1": random.randint(32, 512),
            "neurons2": random.randint(32, 512),
            "learning_rate": 10 ** random.uniform(-4, -2)  # Log-scale
        }
        population.append(individual)
    return population

# Updated model creation for GA
def create_model(neurons1, neurons2, lr):
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),  # Input layer for MNIST
        
        # First hidden layer with BatchNorm, He init, and L2 regularization
        keras.layers.Dense(neurons1, kernel_initializer="he_normal", kernel_regularizer=keras.regularizers.l2(0.01), use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("relu"),
        
        # Second hidden layer
        keras.layers.Dense(neurons2, kernel_initializer="he_normal", kernel_regularizer=keras.regularizers.l2(0.01), use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("relu"),
        
        # Dropout for regularization
        keras.layers.Dropout(rate=0.3),
        
        # Output layer
        keras.layers.Dense(10, activation="softmax", use_bias=False)
    ])
    
    # Compile the model with Adam optimizer and sparse categorical crossentropy loss
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
loss="sparse_categorical_crossentropy", 
metrics=["accuracy"])
    return model

def fitness(individual, X_train, y_train, X_valid, y_valid):
    model = create_model(individual["neurons1"], individual["neurons2"], individual["learning_rate"])
    early_stopping = keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
    
    # Train the model
    history = model.fit(
        X_train, y_train, 
        epochs=5, 
        validation_data=(X_valid, y_valid),
        batch_size=32, 
        verbose=0, 
        callbacks=[early_stopping]
    )
    
    val_loss = history.history["val_loss"][-1]  # Final validation loss
    val_accuracy = history.history["val_accuracy"][-1]  # Final validation accuracy
    
    return val_loss, val_accuracy

# Step 4: Selection (Tournament Selection)
def select_parents(population, fitness_scores):
    parents = random.choices(population, weights=1/np.array(fitness_scores), k=2)  # Lower loss = higher fitness
    return parents

# Step 5: Crossover
def crossover(parent1, parent2):
    child = {
        "neurons1": random.choice([parent1["neurons1"], parent2["neurons1"]]),
        "neurons2": random.choice([parent1["neurons2"], parent2["neurons2"]]),
        "learning_rate": random.choice([parent1["learning_rate"], parent2["learning_rate"]])
    }
    return child

# Step 6: Mutation
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        individual["neurons1"] = random.randint(32, 512)
    if random.random() < mutation_rate:
        individual["neurons2"] = random.randint(32, 512)
    if random.random() < mutation_rate:
        individual["learning_rate"] = 10 ** random.uniform(-4, -2)
    return individual


def genetic_algorithm(X_train, y_train, X_valid, y_valid, generations=10, pop_size=10, mutation_rate=0.1):
    population = initialize_population(pop_size)
    all_fitness_scores = []  # To store all generations' fitness scores
    all_accuracy_scores = []  # To store all generations' accuracy scores

    for generation in range(generations):
        print(f"Generation {generation+1}/{generations}")
        
        # Evaluate fitness for each individual
        fitness_scores = []
        accuracy_scores = []
        for ind in population:
            val_loss, val_accuracy = fitness(ind, X_train, y_train, X_valid, y_valid)
            fitness_scores.append(val_loss)
            accuracy_scores.append(val_accuracy)
        
        # Log best results of the current generation
        best_loss = min(fitness_scores)
        best_accuracy = max(accuracy_scores)
        print(f"Best fitness (val_loss): {best_loss}")
        print(f"Best accuracy (val_accuracy): {best_accuracy}")

        # Store generation metrics
        all_fitness_scores.append(best_loss)
        all_accuracy_scores.append(best_accuracy)

        # Select next generation
        new_population = []
        for _ in range(pop_size // 2):  # Each iteration produces 2 children
            parent1, parent2 = select_parents(population, fitness_scores)
            child1 = mutate(crossover(parent1, parent2), mutation_rate)
            child2 = mutate(crossover(parent1, parent2), mutation_rate)
            new_population.extend([child1, child2])
        
        population = new_population
    
    # Return the best hyperparameters and their performance
    best_index = np.argmin(fitness_scores)
    return population[best_index], all_fitness_scores, all_accuracy_scores
# Rebuild and evaluate the best model
best_hyperparameters = genetic_algorithm(X_train, y_train, X_valid, y_valid, generations=5, pop_size=5, mutation_rate=0.1)
print("Best Hyperparameters:", best_hyperparameters)

# Train the best model on the full training set (train + valid) and evaluate on test set
final_model = create_model(best_hyperparameters["neurons1"], 
best_hyperparameters["neurons2"], 
best_hyperparameters["learning_rate"])

final_model.fit(np.concatenate([X_train, X_valid]), 
                np.concatenate([y_train, y_valid]), 
                epochs=10, 
                batch_size=32)

# Evaluate on the test set
test_loss, test_accuracy = final_model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Generation 1/5


c:\Users\intel\Desktop\ai cep\myenv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best fitness (val_loss): 0.22106555104255676
Best accuracy (val_accuracy): 0.9732000231742859
Generation 2/5
Best fitness (val_loss): 0.18981164693832397
Best accuracy (val_accuracy): 0.9751999974250793
Generation 3/5
Best fitness (val_loss): 0.23137754201889038
Best accuracy (val_accuracy): 0.9718000292778015
Generation 4/5
Best fitness (val_loss): 0.22234196960926056
Best accuracy (val_accuracy): 0.9696000218391418
Generation 5/5
Best fitness (val_loss): 0.2094084918498993
Best accuracy (val_accuracy): 0.9742000102996826
Best Hyperparameters: ({'neurons1': 466, 'neurons2': 302, 'learning_rate': 0.00022667188523287847}, [0.22106555104255676, 0.18981164693832397, 0.23137754201889038, 0.22234196960926056, 0.2094084918498993], [0.9732000231742859, 0.9751999974250793, 0.9718000292778015, 0.9696000218391418, 0.9742000102996826])


TypeError: tuple indices must be integers or slices, not str

In [7]:
import numpy as np
import random
from tensorflow import keras

# Ensure the data is reshaped correctly
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Normalize and split the data
X_train_full, X_test = X_train_full / 255.0, X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_train = X_train.reshape(-1, 28, 28)  # Reshape if necessary
X_valid = X_valid.reshape(-1, 28, 28)  # Reshape if necessary


# Step 1: Define search space
def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        individual = {
            "neurons1": random.randint(32, 512),
            "neurons2": random.randint(32, 512),
            "learning_rate": 10 ** random.uniform(-4, -2)  # Log-scale
        }
        population.append(individual)
    return population

# Step 2: Create the model
def create_model(neurons1, neurons2, lr):
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),  # Adjust input shape for MNIST
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(neurons1, kernel_initializer='lecun_normal', activation='selu'),
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(neurons2, kernel_initializer='lecun_normal', activation='selu'),
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(10, activation="softmax")  # 10 classes for digit classification
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                  loss="sparse_categorical_crossentropy", 
                  metrics=["accuracy"])
    return model


# Step 3: Fitness function
def fitness(individual, X_train, y_train, X_valid, y_valid):
    model = create_model(individual["neurons1"], individual["neurons2"], individual["learning_rate"])
    early_stopping = keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid),
                        batch_size=32, verbose=0, callbacks=[early_stopping])
    
    val_loss = history.history["val_loss"][-1]  # Final validation loss
    val_accuracy = history.history["val_accuracy"][-1]  # Final validation accuracy
    
    return val_loss, val_accuracy  # Both loss and accuracy returned

# Step 4: Selection (Tournament Selection)
def select_parents(population, fitness_scores):
    # Combine loss and accuracy for a balanced selection process
    combined_scores = np.array(fitness_scores)  # Use loss as the main metric for selection
    parents = random.choices(population, weights=1 / combined_scores, k=2)
    return parents

# Step 5: Crossover
def crossover(parent1, parent2):
    child = {
        "neurons1": random.choice([parent1["neurons1"], parent2["neurons1"]]),
        "neurons2": random.choice([parent1["neurons2"], parent2["neurons2"]]),
        "learning_rate": random.choice([parent1["learning_rate"], parent2["learning_rate"]])
    }
    return child

# Step 6: Mutation
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        individual["neurons1"] = random.randint(32, 512)
    if random.random() < mutation_rate:
        individual["neurons2"] = random.randint(32, 512)
    if random.random() < mutation_rate:
        individual["learning_rate"] = 10 ** random.uniform(-4, -2)
    return individual

# Step 7: Genetic Algorithm
def genetic_algorithm(X_train, y_train, X_valid, y_valid, generations, pop_size, mutation_rate):
    population = initialize_population(pop_size)
    
    for generation in range(generations):
        print(f"Generation {generation + 1}/{generations}")
        
        # Evaluate fitness for each individual in the population
        results = [fitness(ind, X_train, y_train, X_valid, y_valid) for ind in population]
        fitness_scores = [result[0] for result in results]  # Extract val_loss
        accuracy_scores = [result[1] for result in results]  # Extract val_accuracy
        
        # Log the best results of the current generation
        best_loss = min(fitness_scores)
        best_accuracy = max(accuracy_scores)
        print(f"Best fitness (val_loss): {best_loss}")
        print(f"Best accuracy (val_accuracy): {best_accuracy}")
        
        # Select next generation
        new_population = []
        for _ in range(pop_size // 2):  # Each iteration produces 2 children
            parent1, parent2 = select_parents(population, fitness_scores)
            child1 = mutate(crossover(parent1, parent2), mutation_rate)
            child2 = mutate(crossover(parent1, parent2), mutation_rate)
            new_population.extend([child1, child2])
        
        population = new_population
    
    # Return the best hyperparameters, loss, and accuracy
    best_index = np.argmin(fitness_scores)  # Best individual based on validation loss
    return population[best_index], best_loss, accuracy_scores[best_index]

# Run Genetic Algorithm
best_hyperparameters, best_loss, best_accuracy = genetic_algorithm(
    X_train, y_train, X_valid, y_valid, 
    generations=5, 
    pop_size=5, 
    mutation_rate=0.1
)

print("Best Hyperparameters:", best_hyperparameters)
print(f"Best Validation Loss: {best_loss}")
print(f"Best Validation Accuracy: {best_accuracy}")


Generation 1/5
Best fitness (val_loss): 0.08322973549365997
Best accuracy (val_accuracy): 0.9750000238418579
Generation 2/5
Best fitness (val_loss): 0.08720405399799347
Best accuracy (val_accuracy): 0.9753999710083008
Generation 3/5
Best fitness (val_loss): 0.0769028589129448
Best accuracy (val_accuracy): 0.9764000177383423
Generation 4/5
Best fitness (val_loss): 0.08446028083562851
Best accuracy (val_accuracy): 0.9775999784469604
Generation 5/5
Best fitness (val_loss): 0.07584147900342941
Best accuracy (val_accuracy): 0.9775999784469604
Best Hyperparameters: {'neurons1': 472, 'neurons2': 101, 'learning_rate': 0.00041933307406293475}
Best Validation Loss: 0.07584147900342941
Best Validation Accuracy: 0.9775999784469604
